In [2]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0, VGG16, ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# Define the paths to your dataset folders
train_data_dir = '/kaggle/input/urecamain/Train'
validation_data_dir = '/kaggle/input/urecamain/Vali'
test_data_dir = '/kaggle/input/urecamain/Test'

# Set up data generators for training, validation, and testing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'  # Assuming two classes: fire and no fire
)

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

# Load the pretrained models
efficientnet_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
vgg16_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
resnet50_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add classification layers on top of each model
model_efficientnet = Sequential([
    efficientnet_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')  # Binary classification for fire detection
])

model_vgg16 = Sequential([
    vgg16_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])

model_resnet50 = Sequential([
    resnet50_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Create separate optimizer instances for each model
optimizer_efficientnet = Adam(learning_rate=0.001)
optimizer_vgg16 = Adam(learning_rate=0.001)
optimizer_resnet50 = Adam(learning_rate=0.001)

# Compile the models with their respective optimizers
loss_function = 'binary_crossentropy'
metrics = ['accuracy']

model_efficientnet.compile(optimizer=optimizer_efficientnet, loss=loss_function, metrics=metrics)
model_vgg16.compile(optimizer=optimizer_vgg16, loss=loss_function, metrics=metrics)
model_resnet50.compile(optimizer=optimizer_resnet50, loss=loss_function, metrics=metrics)

# Train the models
epochs = 10
model_efficientnet.fit(train_generator, epochs=epochs, validation_data=validation_generator)
model_vgg16.fit(train_generator, epochs=epochs, validation_data=validation_generator)
model_resnet50.fit(train_generator, epochs=epochs, validation_data=validation_generator)

# Evaluate the models on the test set
eval_efficientnet = model_efficientnet.evaluate(test_generator)
eval_vgg16 = model_vgg16.evaluate(test_generator)
eval_resnet50 = model_resnet50.evaluate(test_generator)

# Compare the models' performance
best_accuracy = max(eval_efficientnet[1], eval_vgg16[1], eval_resnet50[1])
best_model = None

if best_accuracy == eval_efficientnet[1]:
    best_model = 'EfficientNetB0'
elif best_accuracy == eval_vgg16[1]:
    best_model = 'VGG16'
else:
    best_model = 'ResNet50'

print("Best Model:", best_model)
print("Best Accuracy:", best_accuracy)


Found 6003 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Epoch 1/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 199s 660ms/step - accuracy: 0.9230 - loss: 0.1778 - val_accuracy: 0.5000 - val_loss: 8.2503
Epoch 2/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 80s 409ms/step - accuracy: 0.9779 - loss: 0.0648 - val_accuracy: 0.5000 - val_loss: 2.7984
Epoch 3/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 79s 406ms/step - accuracy: 0.9790 - loss: 0.0543 - val_accuracy: 0.5000 - val_loss: 3.6582
Epoch 4/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 79s 405ms/step - accuracy: 0.9839 - loss: 0.0494 - val_accuracy: 0.6550 - val_loss: 1.4523
Epoch 5/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 79s 408ms/step - accuracy: 0.9916 - loss: 0.0236 - val_accuracy: 0.8470 - val_loss: 0.5256
Epoch 6/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 79s 406ms/step - accuracy: 0.9913 - loss: 0.0257 - val_accuracy: 0.9335 - val_loss: 0.2536
Epoch 8/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 80s 410ms/step - accuracy: 0.9880 - loss: 0.043

2024-04-13 07:02:51.183838: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 0: 4.63498, expected 3.85968
2024-04-13 07:02:51.183898: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 3: 6.55784, expected 5.78254
2024-04-13 07:02:51.183907: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 4: 6.61282, expected 5.83752
2024-04-13 07:02:51.183915: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 6: 6.34499, expected 5.56968
2024-04-13 07:02:51.183923: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 7: 6.25159, expected 5.47629
2024-04-13 07:02:51.183931: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 8: 5.43964, expected 4.66434
2024-04-13 07:02:51.183939: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 9: 6.54333, expected 5.76803
2024-04-13 07:02:51.183946: E external/local_xla/xla/se

 66/188 ━━━━━━━━━━━━━━━━━━━━ 48s 399ms/step - accuracy: 0.4842 - loss: 10.2187

2024-04-13 07:03:52.434649: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 0: 3.83445, expected 3.08393
2024-04-13 07:03:52.434717: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 1: 5.81387, expected 5.06335
2024-04-13 07:03:52.434726: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 2: 5.65478, expected 4.90426
2024-04-13 07:03:52.434734: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 3: 6.2384, expected 5.48788
2024-04-13 07:03:52.434742: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 4: 5.85899, expected 5.10847
2024-04-13 07:03:52.434750: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 5: 5.234, expected 4.48348
2024-04-13 07:03:52.434758: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 6: 5.17387, expected 4.42335
2024-04-13 07:03:52.434765: E external/local_xla/xla/servi

188/188 ━━━━━━━━━━━━━━━━━━━━ 0s 517ms/step - accuracy: 0.4913 - loss: 1527.7704

2024-04-13 07:05:07.434260: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 0: 3.89842, expected 3.37692
2024-04-13 07:05:07.434330: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 12: 3.13412, expected 2.61262
2024-04-13 07:05:07.434341: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 13: 3.08377, expected 2.56227
2024-04-13 07:05:07.434349: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 14: 4.1021, expected 3.5806
2024-04-13 07:05:07.434358: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 29: 3.76539, expected 3.24388
2024-04-13 07:05:07.434376: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 30: 4.19841, expected 3.6769
2024-04-13 07:05:07.434385: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 63: 3.88366, expected 3.36216
2024-04-13 07:05:07.434393: E external/local_xla/xla

188/188 ━━━━━━━━━━━━━━━━━━━━ 153s 592ms/step - accuracy: 0.4914 - loss: 1527.4287 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 2/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 82s 420ms/step - accuracy: 0.5047 - loss: 262453.4062 - val_accuracy: 0.5000 - val_loss: 0.8095
Epoch 3/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 81s 413ms/step - accuracy: 0.5558 - loss: 9954102607872.0000 - val_accuracy: 0.4605 - val_loss: 149.1592
Epoch 5/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 81s 413ms/step - accuracy: 0.5550 - loss: 49346728951808.0000 - val_accuracy: 0.7770 - val_loss: 0.5112
Epoch 6/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 82s 418ms/step - accuracy: 0.5055 - loss: 5787911979008.0000 - val_accuracy: 0.5200 - val_loss: 62.5909
Epoch 7/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 81s 413ms/step - accuracy: 0.5000 - loss: 3113211920384.0000 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 8/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 80s 410ms/step - accuracy: 0.4906 - loss: 158043406336.0000 - val_accuracy: 0.5000 - val_loss: 3608852.7500
Epoch 9/10
188/1